In [1]:
import pandas as pd
import numpy as np

# Using apply() and groupby() to create your own groups

Here we will show a simple use of apply() method and groupby() method that can be very useful. 

Let us say you want to group the 'AIRLINE' but not necessarily the same airline but according to the their alliance. How can you achieve this? THe following are the groups of airlines that are in each of the alliances

Star Alliance:
* UA - United Airlines
* OO - Skywest Airlines

Oneworld Alliance:
* AA - American Airlines
* US - US Airlines
* MQ - American Eagle Airlines Inc. 

SkyTeam Alliance:
* DL - Delta Airlines
* EV - Atlantic Sotheast Airlines
* VX - Virgin America

NoAlliance; Not in any alliance:

* F9 - Forntier
* B6 - Jetblue
* NK - Spirit
* WN - Southwest
* HA - Hawaiian
* AS - Alaskan 


In [2]:
flights = pd.read_csv('./data/flight_sample.csv')
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAXI_IN,TAXI_OUT,DISTANCE
0,2015,8,19,3,EV,3260,7.0,20.0,1091
1,2015,9,23,3,WN,3050,4.0,9.0,837
2,2015,10,16,5,AA,1382,13.0,25.0,761
3,2015,1,19,1,WN,4274,5.0,23.0,1547
4,2015,4,22,3,WN,2237,5.0,18.0,872


In [3]:
def get_alliance(airline):
    if airline in ['UA','OO']:
        return 'Star'
    elif airline in ['AA', 'US', 'MQ']:
        return 'Oneworld'
    elif airline in ['DL','EV', 'VX']:
        return 'SkyTeam'
    elif airline in ['AS', 'F9', 'B6', 'NK', 'WN', 'HA']:
        return 'NoAlliance'

In [5]:
get_alliance('WN')

'NoAlliance'

In [6]:
get_alliance('UA')

'Star'

In [7]:
# CREATING a new column called 'Alliance' and assigning the alliance based on the function
flights['Alliance'] = flights['AIRLINE'].apply(get_alliance)

**NOTE**: We only pass the name of the function ``get_alliance`` to the apply method. Internally, the ``get_alliance`` function is called (applied) on each value of the ``flights['AIRLINE']`` column. 

In [8]:
flights.sample(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAXI_IN,TAXI_OUT,DISTANCE,Alliance
1686,2015,2,5,4,DL,1245,4.0,17.0,1013,SkyTeam
4649,2015,7,9,4,OO,7368,10.0,14.0,306,Star
4674,2015,5,26,2,DL,1838,8.0,27.0,547,SkyTeam
4392,2015,12,9,3,MQ,3337,6.0,15.0,135,Oneworld
775,2015,10,2,5,DL,2035,8.0,19.0,669,SkyTeam


In [9]:
flights_by_alliance = flights.groupby(['Alliance'])

In [10]:
type(flights_by_alliance)

pandas.core.groupby.generic.DataFrameGroupBy

In [11]:
df_means = flights_by_alliance[['DISTANCE','TAXI_IN']].mean()

In [12]:
df_means

,DISTANCE,TAXI_IN
Alliance,,
NoAlliance,843.074474,6.593195
Oneworld,882.712241,8.769113
SkyTeam,734.778623,7.450596
Star,854.930765,7.617179


In [13]:
type(df_means)

pandas.core.frame.DataFrame

In [14]:
df_means.loc['Star']['DISTANCE']

854.9307652264446

### Detour:  `sort_values()`, a method to sort rows based on a column

In [15]:
df_means

,DISTANCE,TAXI_IN
Alliance,,
NoAlliance,843.074474,6.593195
Oneworld,882.712241,8.769113
SkyTeam,734.778623,7.450596
Star,854.930765,7.617179


In [16]:
df_means.sort_values(['DISTANCE'], inplace=True)
df_means

,DISTANCE,TAXI_IN
Alliance,,
SkyTeam,734.778623,7.450596
NoAlliance,843.074474,6.593195
Star,854.930765,7.617179
Oneworld,882.712241,8.769113


### Activity:

1. Drop all rows in `college_scorecard_small` that has any missing values

2. Add another column `sat_avg_level` to the `college_scorecard_small` DataFrame. It is assigned the following values based on the values in `sat_average`. **You need to write a function and use ``apply()`` method**
   * Lower_sat
       - sat_average <= 973 
   * Below_avg_sat
       - 973 < sat_average <= 1039
   * Abv_avg_sat
       - 1039 < sat_average <= 1120
   * Higher_sat
       - sat_average> 1120
   
3. Is there a relationship between `sat_avg_level` and `pell_grant_receipents`? How about relationship with `full_time_retention_rate_4_year`? 
   * Group by sat_avg_level and find the average for the rest of the two columns and make your interpretation. 

In [17]:
college_scorecard = pd.read_csv(
    './data/college-scorecard-data-scrubbed.csv', 
    encoding='latin-1')


# I'm extracting only three columns and creating a copy for this analysis. 
college_scorecard_small = college_scorecard[['sat_average', 'pell_grant_receipents','full_time_retention_rate_4_year']].copy()
college_scorecard_small.head()

,sat_average,pell_grant_receipents,full_time_retention_rate_4_year
0,NaN,0.3571,0.3333
1,NaN,0.7078,NaN
2,NaN,0.8868,NaN
3,1054.0,0.3152,0.7742
4,NaN,0.0737,NaN


In [18]:
# 1. Drop the rows with missing values

college_scorecard_small.dropna(inplace=True)

# Second way

# college_scorecard_small = college_scorecard_small.dropna()

In [ ]:
# Lower_sat
# sat_average <= 973
# Below_avg_sat
# 973 < sat_average <= 1039
# Abv_avg_sat
# 1039 < sat_average <= 1120
# Higher_sat
# sat_average> 1120

In [19]:
def get_sat_level(sat_avg):
    if sat_avg <= 973:
        return 'Lower_sat'
    elif 973 < sat_avg <= 1039:
        return 'Below_avg_sat'
    elif 1039 < sat_avg <= 1120:
        return 'Abv_avg_sat'
    else:
        return 'Higher_sat'

In [21]:
college_scorecard_small['sat_avg_level'] = college_scorecard_small['sat_average'].apply(get_sat_level)
college_scorecard_small.head()

,sat_average,pell_grant_receipents,full_time_retention_rate_4_year,sat_avg_level
3,1054.0,0.3152,0.7742,Abv_avg_sat
10,827.0,0.7356,0.6595,Lower_sat
14,851.0,0.7347,0.5725,Lower_sat
17,1215.0,0.1610,0.9052,Higher_sat
18,1009.0,0.4192,0.6520,Below_avg_sat


In [22]:
college_by_sat = college_scorecard_small.groupby(['sat_avg_level'])
avg_pell_fulltime = college_by_sat[['pell_grant_receipents', 'full_time_retention_rate_4_year']].mean()
avg_pell_fulltime

,pell_grant_receipents,full_time_retention_rate_4_year
sat_avg_level,,
Abv_avg_sat,0.342678,0.776322
Below_avg_sat,0.402556,0.716375
Higher_sat,0.236075,0.882165
Lower_sat,0.491596,0.671799


In [23]:
avg_pell_fulltime.sort_values(['pell_grant_receipents'])

,pell_grant_receipents,full_time_retention_rate_4_year
sat_avg_level,,
Higher_sat,0.236075,0.882165
Abv_avg_sat,0.342678,0.776322
Below_avg_sat,0.402556,0.716375
Lower_sat,0.491596,0.671799


In [24]:
avg_pell_fulltime.sort_values(['full_time_retention_rate_4_year'])

,pell_grant_receipents,full_time_retention_rate_4_year
sat_avg_level,,
Lower_sat,0.491596,0.671799
Below_avg_sat,0.402556,0.716375
Abv_avg_sat,0.342678,0.776322
Higher_sat,0.236075,0.882165


# Pivot Tables: Two-dimensional GroupBy

We have seen how the ``GroupBy`` abstraction lets us explore relationships within a dataset.
A *pivot table* is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data.
The pivot table takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.

In [25]:
# index is the row grouping and columns is the column grouping, the first parameter is the one that is aggregated

flight_pvt = flights.pivot_table('DISTANCE',index='DAY_OF_WEEK', columns = 'Alliance')
flight_pvt

Alliance,NoAlliance,Oneworld,SkyTeam,Star
DAY_OF_WEEK,,,,
1,839.861856,841.852113,785.222772,871.846154
2,857.928155,817.296167,696.713178,809.895911
3,811.619145,894.945848,765.838791,852.893993
4,823.606238,874.069079,727.983491,816.889262
5,861.238281,922.049853,701.717391,881.644195
6,853.011765,941.658537,723.755853,880.654762
7,855.118012,888.644599,735.820000,875.887218


### Modifying the defualt behavior with `aggfunc` keyword argument

In [26]:
flight_pvt = flights.pivot_table('DISTANCE',index='DAY_OF_WEEK', columns = 'Alliance', aggfunc = np.sum)
flight_pvt

Alliance,NoAlliance,Oneworld,SkyTeam,Star
DAY_OF_WEEK,,,,
1,407333,239086,317230,249348
2,441833,234564,269628,217862
3,398505,247900,304038,241369
4,422510,265717,308665,243433
5,440954,314419,258232,235399
6,362530,231648,216403,221925
7,413022,255041,257537,232986


### You can get the totals using `margins` keyword argument

In [33]:
flight_pvt = flights.pivot_table('DISTANCE',index='DAY_OF_WEEK', columns = 'Alliance', aggfunc = np.median, margins=True)
flight_pvt

Alliance,NoAlliance,Oneworld,SkyTeam,Star,All
DAY_OF_WEEK,,,,,
1,711.0,688.0,534.0,733.0,661.0
2,738.0,678.0,547.0,616.0,632.0
3,674.0,742.0,591.0,637.0,650.0
4,721.0,735.0,549.0,624.5,642.0
5,704.0,802.0,530.0,710.0,653.0
6,804.0,867.0,557.0,661.5,690.5
7,680.0,733.0,577.0,678.5,660.0
All,717.0,733.0,554.0,657.0,651.0


In [35]:
flight_pvt = flights.pivot_table('DISTANCE',index='DAY_OF_WEEK', columns = 'Alliance', aggfunc = np.sum, margins=True)
flight_pvt

Alliance,NoAlliance,Oneworld,SkyTeam,Star,All
DAY_OF_WEEK,,,,,
1,407333,239086,317230,249348,1212997
2,441833,234564,269628,217862,1163887
3,398505,247900,304038,241369,1191812
4,422510,265717,308665,243433,1240325
5,440954,314419,258232,235399,1249004
6,362530,231648,216403,221925,1032506
7,413022,255041,257537,232986,1158586
All,2886687,1788375,1931733,1642322,8249117


In [36]:
flights['DISTANCE'].sum()

8249117

In [37]:
flight_pvt.loc[2]['NoAlliance']

441833

### Pivot table vs. Group By

They are two sides of the same coin. Pivot table looks clean and neat, however, you can achieve the same thing using groupby on two columns. 

In [38]:
flights_by_dayweek_alliance = flights.groupby(['DAY_OF_WEEK','Alliance'])
flights_by_dayweek_alliance[['DISTANCE']].sum()

DISTANCE
DAY_OF_WEEK Alliance            
1           NoAlliance    407333
            Oneworld      239086
            SkyTeam       317230
            Star          249348
2           NoAlliance    441833
            Oneworld      234564
            SkyTeam       269628
            Star          217862
3           NoAlliance    398505
            Oneworld      247900
            SkyTeam       304038
            Star          241369
4           NoAlliance    422510
            Oneworld      265717
            SkyTeam       308665
            Star          243433
5           NoAlliance    440954
            Oneworld      314419
            SkyTeam       258232
            Star          235399
6           NoAlliance    362530
            Oneworld      231648
            SkyTeam       216403
            Star          221925
7           NoAlliance    413022
            Oneworld      255041
            SkyTeam       257537
            Star          232986

## Activity: Birthrate Data

As a more interesting example, let's take a look at the freely available data on births in the United States, provided by the Centers for Disease Control (CDC).
This data can be found at https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv
(this dataset has been analyzed rather extensively by Andrew Gelman and his group; see, for example, [this blog post](http://andrewgelman.com/2012/06/14/cool-ass-signal-processing-using-gaussian-processes/)):

1. Create a column called `decade` in the births_df dataframe loaded below. 
    * Use the column called `year` to create the `decade`. For example, if you were born in 1969 then the decade is 1960
2. Create a pivot table that counts number of `births` in each decade and also based on weather they were male or female
3. Also create an equivalent groupby that produces the results similar to pivot table. 

In [39]:
births_df = pd.read_csv('./data/births.csv')

In [40]:
births_df.head()

,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548


In [45]:
(1975//10)*10

1970

In [46]:
def get_decade(year):
    return (year//10)*10

In [47]:
get_decade(1984)

1980

In [48]:
births_df['decade'] = births_df['year'].apply(get_decade)
births_df.sample(5)

,year,month,day,gender,births,decade
13249,1986,7,14.0,F,5364,1980
8978,1980,10,18.0,F,4298,1980
8907,1980,9,13.0,M,4950,1980
9240,1981,2,22.0,M,4382,1980
2172,1971,11,1.0,F,4596,1970


In [54]:
births_pvt = births_df.pivot_table('births', index = 'decade', columns = 'gender', aggfunc=np.sum)
births_pvt

gender,F,M
decade,,
1960,1753634,1846572
1970,16263075,17121550
1980,18310351,19243452
1990,19479454,20420553
2000,18229309,19106428


In [55]:
# Decade with most number of female births
births_pvt['F'].idxmax()

1990

<div class="alert alert-block alert-danger">
<h3> CAUTION AHEAD </h3>
<p> </p>
<p> The topics discussed ahead are advanced and you need to absolutely make sure you understand everything discussed in the previous classes to move forward.</p>
</div>

### Advanced Topics: filter() and transform()

These functions give a lot more flexibility on `DataFrameGroupBY` objects and they are discussed below. They are advanced topics, however, I **strongly encourage** you to read through them and you could use them for finding very interesting patterns in the data. 

In [56]:
# Starter code for the Advanced Topics, you will need to run this before you use them further. 
college_loan_defaults = pd.read_csv(
    './data/college-loan-default-rates.csv')

college_scorecard = pd.read_csv(
    './data/college-scorecard-data-scrubbed.csv', 
    encoding='latin-1')



#### The `filter()` Method

You can use the `filter()` method to generate a new dataframe after filtering out groups that don't pass a given criteria. It allows you to answer questions like this: *what states in college scorecard have rows where the average SAT score (for the state) is above 1100?*

To use this method, you must pass in a function that takes a single parameter, which is the group to evaluate. The function must return either `True`/`False` depending on whether or not the *rows of the group* should be kept or discarded in the new dataframe.

So, with this in mind, let's define a `sat_filter` function so that groups with average SAT scores of less than 1100 are dropped from consideration.

In [57]:
# Here I'm just extracting the four columns that I'm interested in
college_scorecard_small = college_scorecard[['institution_name','state','city', 'sat_average']]

In [58]:
college_scorecard_small.head()

,institution_name,state,city,sat_average
0,Alaska Bible College,AK,Palmer,NaN
1,Alaska Career College,AK,Anchorage,NaN
2,Alaska Christian College,AK,Soldotna,NaN
3,Alaska Pacific University,AK,Anchorage,1054.0
4,AVTEC-Alaska's Institute of Technology,AK,Seward,NaN


In [59]:
colleges_by_state = college_scorecard_small.groupby(['state'])

In [60]:
def size_filter(group):
    if group['sat_average'].mean() >= 1150:
        return True
    else:
        return False

And now let's use it on to see which rows remain in the new dataframe after applying the filter:

In [62]:
filter_results = colleges_by_state.filter(size_filter)
filter_results['state'].unique()

array(['DC', 'RI'], dtype=object)

In [63]:
filter_results

,institution_name,state,city,sat_average
1246,American University,DC,Washington,1252.0
1247,Bennett Career Institute,DC,Washington,NaN
1248,Career Technical Institute,DC,Washington,NaN
1249,Catholic University of America,DC,Washington,1130.0
1250,Gallaudet University,DC,Washington,849.0
1251,George Washington University,DC,Washington,1297.0
1252,Georgetown University,DC,Washington,1414.0
1253,Graduate School USA,DC,Washington,NaN
1254,Howard University,DC,Washington,1105.0
1255,Institute of World Politics,DC,Washington,NaN


There are a couple of ***really*** important things to notice here:
1. Unlike the **`aggregate`** method, the data returned here is not grouped by state as you probably expected it to be. The filter is used on a grouped dataframe, but it returns a new "normal" dataframe.
2. Notice that we have a bunch of rows for Washington DC and Rhode Island, but nothing else. If we've done things correctly, this would mean that the colleges in those two states have average SAT scores of at least 1150. 

#### The `transform()` Method

You use the **`transform()`** method to generate a new dataframe that modifies/transforms the values of the grouped dataframes columns.

That probably just confused the heck out of you. So we will start with a practical example.

Let's say that we wanted to center the data for the *`year_1_default_rate`* and *`year_2_default_rate`* columns of our **`college_loan_defaults_by_state`** grouped dataframe. 

Let's step through how we could do that with **`transform()`**.

Just like with the **`filter()`** method, we have to create a function that we will pass to the **`transform`** method, but this time the function will evaluate each series (column) of each group, rather than the groups as a whole.

In [64]:
# Just extracting three columns for this analysis
college_loan_defaults_subset = college_loan_defaults[['name', 'state', 'year_1_default_rate']]
college_loan_defaults_subset.head()

,name,state,year_1_default_rate
0,A - TECHNICAL COLLEGE,CA,27.1
1,A & W HEALTHCARE EDUCATORS,LA,12.9
2,A. T. STILL UNIVERSITY OF HEALTH SCIENCES,MO,1.6
3,AARON'S ACADEMY OF BEAUTY,MD,35.8
4,ABC BEAUTY COLLEGE,AR,26.6


In [65]:
college_loan_defaults_by_state = college_loan_defaults_subset.groupby(['state'])

In [66]:
# This function will be called on each 
# series of each group in your DataFrameGroupBy object
def center_default_rate(series):
    return series - series.mean()

In [67]:
transformed_default_rates = college_loan_defaults_by_state.transform(center_default_rate)
transformed_default_rates.head()

,year_1_default_rate
0,16.679912
1,0.786567
2,-9.209489
3,24.831818
4,12.159375


In [68]:
# We'll also use the rename() method to apply some friendly column names.
transformed_default_rates = college_loan_defaults_by_state.transform(
    center_default_rate).rename(
        columns={'year_1_default_rate': 'centered_year_1_default_rate'})
transformed_default_rates.head()

,centered_year_1_default_rate
0,16.679912
1,0.786567
2,-9.209489
3,24.831818
4,12.159375


<div class="alert alert-block alert-info">
<p>
Our `college_loan_defaults_by_state` dataframe included four columns: name, state, and year_1_default_rate.
</p> 
<p>But here in the returned dataframe we only have `centered_year_1_default_rate`. The reason for this is that the other two columns were strings, and you can't calculate the mean of a series of strings.
</p>
<p>
Because of this, Pandas just silently drops them from the new dataframe that is returned from the `tranform` method.
</p>
</div>

So now we have our centered rates in a new dataframe. Let's merge together the result of our **`transform`** method and our *`college_loan_defaults_subset`* dataframe. 

In [69]:
# Make sure to specify the indices as the "join column" or Pandas
# will try to join the dataframes based on the shared 'year_1_default_rate' column.
pd.merge(college_loan_defaults_subset, transformed_default_rates, 
         left_index=True, right_index=True)[:5]

,name,state,year_1_default_rate,centered_year_1_default_rate
0,A - TECHNICAL COLLEGE,CA,27.1,16.679912
1,A & W HEALTHCARE EDUCATORS,LA,12.9,0.786567
2,A. T. STILL UNIVERSITY OF HEALTH SCIENCES,MO,1.6,-9.209489
3,AARON'S ACADEMY OF BEAUTY,MD,35.8,24.831818
4,ABC BEAUTY COLLEGE,AR,26.6,12.159375
